In [2]:
# Install necessary packages
!pip install torch torchvision

# Clone the YOLOv10 repository
!git clone https://github.com/THU-MIG/yolov10.git
%cd yolov10

# Install required dependencies
!pip install -r requirements.txt


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Cloning into 'yolov10'...
remote: Enumerating objects: 20304, done.
remote: Counting objects: 100% (1502/1502), done.
remote: Compressing objects: 100% (154/154), done.
error: RPC failed; curl 18 Transferred a partial file.00 KiB/s  
error: 6760 bytes of body are still expected
fetch-pack: unexpected disconnect while reading sideband packet
fatal: early EOF
fatal: fetch-pack: invalid index-pack output
[Errno 2] No such file or directory: 'yolov10'
/Users/apple/Desktop/PG/Summer-24/image-DL/knee-arthritis-detection-algo


/Users/apple/Library/Python/3.9/lib/python/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
import os
import shutil
from PIL import Image
import yaml
from sklearn import transforms datasets 


# Directory paths
dataset_path = '/Users/apple/Desktop/PG/Summer-24/image-DL/knee-arthritis-detection-algo/Training'
train_path = os.path.join(dataset_path, 'train')
val_path = os.path.join(dataset_path, 'val')
labels_path = os.path.join(dataset_path, 'labels')

# Create necessary directories
os.makedirs(train_path, exist_ok=True)
os.makedirs(val_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

# Define transformations
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
])

# Custom Dataset class to handle image loading and preprocessing
class KneeDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, img) for img in os.listdir(root_dir) if img.endswith('.png')]
        self.labels = [0 if '0Normal' in img else 1 for img in os.listdir(root_dir) if img.endswith('.png')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Prepare the dataset
train_dataset = KneeDataset(train_path, transform=transform)
val_dataset = KneeDataset(val_path, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

# Define the labels.yaml file required for YOLOv10
labels_yaml = {
    'train': train_path,
    'val': val_path,
    'nc': 2,
    'names': ['Normal', 'Arthritis']
}

# Save the labels.yaml file
with open(os.path.join(labels_path, 'labels.yaml'), 'w') as file:
    yaml.dump(labels_yaml, file)


NameError: name 'transforms' is not defined

In [ ]:
import torch
from yolov10 import YOLOv10

# Load YOLOv10 model
model = YOLOv10('yolov10.pt')

# Define configurations
model.yaml['nc'] = 2
model.yaml['names'] = ['Normal', 'Arthritis']


In [ ]:
# Train the model
model.train(data=os.path.join(labels_path, 'labels.yaml'), epochs=50, batch_size=16)


In [ ]:
# Evaluate the model
results = model.val()
print(results)

# Save the best model
model.save('best_yolov10_model.pth')
